In [1]:
import pandas as pd
import numpy as np
import os
import urllib.request  # lib that handles URLs
import csv

Download pesticide data and convert to csv

In [ ]:
# years = ['1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012']
# for x in years:
#     file = f'EPest.county.estimates.{x}'
#     filecsv = f'{file}.csv'
#     url = f"https://water.usgs.gov/nawqa/pnsp/usage/maps/county-level/PesticideUseEstimates/{file}.txt"
#     urlretrieve(url, f'{file}.txt')
    
#     df = pd.read_csv(f'{file}.txt', sep='\t',encoding='utf-8')

#     filecsv = f'{file}.csv'
#     df.to_csv(f'C:\\Users\\lkb32\\Desktop\\CSE6242\\Project\\data\\Pesticide\\{filecsv}', index=False)

Combine all csvs in each folder into one input csv

In [ ]:
# folder_path1 = r'C:\Users\lkb32\Desktop\CSE6242\Project\data\All_Types'
# folder_path2 = r'C:\Users\lkb32\Desktop\CSE6242\Project\data\Liver'
# folder_path3 = r'C:\Users\lkb32\Desktop\CSE6242\Project\data\Stomach'
#folder_path4 = r'C:\Users\lkb32\Desktop\CSE6242\Project\data\Pesticide'

In [ ]:
# def combine_csv_files(directory, output):
#     all_filenames = [f for f in os.listdir(directory) if f.endswith('.csv')]
#     combined_csv = pd.concat([pd.read_csv(os.path.join(directory, f)) for f in all_filenames])
#     combined_csv.to_csv(output, index=False, encoding='utf-8-sig')

In [ ]:
# combine_csv_files(folder_path1, 'data\combined_all.csv')
# combine_csv_files(folder_path2, 'data\combined_liver.csv')
# combine_csv_files(folder_path3, 'data\combined_stomach.csv')
#combine_csv_files(folder_path4, 'data/combined_pest.csv')

Read in data

In [3]:
datAll = pd.read_csv('data/combined_all.csv') #cancer data 1980 -2014
datLiver = pd.read_csv('data/combined_liver.csv')
datStomach = pd.read_csv('data/combined_stomach.csv')
codes = pd.read_csv('raw_zip_data/IHME_USA_LIVER_CANCER_COUNTY_RACE_ETHN_2000_2019_CODEBOOK_Y2024M02D08.csv')
datPestEst = pd.read_csv('data/EPest_county_estimates_2013_2017_v2.csv')
datPest2 = pd.read_csv('data/combined_pest.csv')

In [ ]:
#datAll = pd.read_csv('data/combined_all.csv')

In [7]:
#combining the for cleaning
#datPest = pd.concat([datPest2,datPestEst)

In [9]:
datPest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8148801 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   COMPOUND          object 
 1   YEAR              int64  
 2   STATE_FIPS_CODE   int64  
 3   COUNTY_FIPS_CODE  int64  
 4   EPEST_LOW_KG      float64
 5   EPEST_HIGH_KG     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 435.2+ MB


In [13]:
datPest.head()

,COMPOUND,YEAR,STATE_FIPS_CODE,COUNTY_FIPS_CODE,EPEST_LOW_KG,EPEST_HIGH_KG
0,"2,4-D",1992,1,1,233.3,913.8
1,"2,4-D",1992,1,3,2895.2,3821.3
2,"2,4-D",1992,1,5,1114.7,1836.2
3,"2,4-D",1992,1,7,404.3,882.0
4,"2,4-D",1992,1,9,1276.5,1568.2


In [15]:
datPest['STATE_FIPS_CODE'] = datPest['STATE_FIPS_CODE'].astype(str)
datPest['COUNTY_FIPS_CODE'] = datPest['COUNTY_FIPS_CODE'].astype(str)
datPest['FIPS'] = datPest['STATE_FIPS_CODE'].str.cat(datPest['COUNTY_FIPS_CODE'], sep = '')
datPest = datPest.drop(columns=['STATE_FIPS_CODE','COUNTY_FIPS_CODE'])

In [ ]:
#which pesticides are included
#datPest['COMPOUND'].unique()

In [17]:
datPestP = datPest.pivot_table(
    index=["YEAR", "FIPS"], 
    columns="COMPOUND", 
    values=["EPEST_LOW_KG", "EPEST_HIGH_KG"]
)
datPestP = datPestP.reset_index()

In [19]:
#replace NaN with 0 
#if no data, we will assume that the level was 0 for this modeling
datPestP = datPestP.fillna(0)

In [21]:
# Flatten the MultiIndex columns
datPestP.columns = ['_'.join(col).strip() for col in datPestP.columns.values]
# Rename the first two columns 
datPestP = datPestP.rename(columns={'YEAR_': 'YEAR', 'FIPS_': 'FIPS'})

In [23]:
datPestP.head()

,YEAR,FIPS,EPEST_HIGH_KG_1-METHYL CYCLOPROPENE,"EPEST_HIGH_KG_2,4-D","EPEST_HIGH_KG_2,4-DB",EPEST_HIGH_KG_6-BENZYLADENINE,EPEST_HIGH_KG_ABAMECTIN,EPEST_HIGH_KG_ABSCISIC ACID,EPEST_HIGH_KG_ACEPHATE,EPEST_HIGH_KG_ACEQUINOCYL,...,EPEST_LOW_KG_TRINEXAPAC,EPEST_LOW_KG_TRITICONAZOLE,EPEST_LOW_KG_UNICONAZOLE,EPEST_LOW_KG_VERNOLATE,EPEST_LOW_KG_VINCLOZOLIN,EPEST_LOW_KG_ZETA-CYPERMETHRIN,EPEST_LOW_KG_ZINC,EPEST_LOW_KG_ZINEB,EPEST_LOW_KG_ZIRAM,EPEST_LOW_KG_ZOXAMIDE
0,1992,101,0.0,10134.2,437.9,0.0,0.0,0.0,348.8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1992,103,0.0,2812.9,720.2,0.0,0.0,0.0,209.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.6,0.0
2,1992,105,0.0,22793.0,578.9,0.0,0.0,0.0,763.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1992,11,0.0,913.8,31.4,0.0,0.0,0.0,188.8,0.0,...,0.0,0.0,0.0,16.8,1.1,9.6,0.0,0.0,0.0,0.0
4,1992,1101,0.0,1364.5,73.6,0.0,0.0,0.0,289.4,0.0,...,0.0,0.0,0.0,2.1,0.0,1.4,0.0,0.0,0.0,0.0


In [25]:
datAll['FIPS'] = pd.to_numeric(datAll['FIPS'], errors='coerce')
# Drop rows with NaN or infinite values in the 'FIPS' column
datAll = datAll.dropna(subset=['FIPS'])
datAll = datAll[np.isfinite(datAll['FIPS'])]
# Convert 'FIPS' column to string
datAll['FIPS'] = datAll['FIPS'].astype(np.int64)  
datAll['FIPS'] = datAll['FIPS'].astype(str)  
datAll['YEAR'] = datAll['year_id'].astype(int)
datAll = datAll.drop(columns=['location_id','location_name','cause_id', 'year_id'])

In [27]:
datAll.head()

,FIPS,cause_name,sex_id,sex,mx,upper,lower,YEAR
0,1,Neoplasms,1,Male,357.835451,368.489566,347.343556,1980
1,1,Neoplasms,1,Male,355.090053,364.802561,343.964164,1981
2,1,Neoplasms,1,Male,350.125517,360.659965,339.142397,1982
3,1,Neoplasms,1,Male,347.808292,357.772762,336.877271,1983
4,1,Neoplasms,1,Male,351.763010,362.472964,340.805798,1984


In [ ]:
#Add in 5-year and 10-year out
# datPest['5_YEAR'] = datPest['YEAR'] +5
# datPest['10_YEAR'] = datPest['YEAR'] +10

In [29]:
#datCombo = datAll.merge(datPestP, on=['YEAR', 'FIPS'])

In [51]:
#datCombo.to_csv(f'C:\\Users\\lkb32\\Desktop\\CSE6242\\Project\\data\\Pesticide\\pesticide_cancer_all.csv', index=False)
datPestP.to_csv(f'C:\\Users\\lkb32\\Desktop\\CSE6242\\Project\\data\\pesticide_pivot_all_cleaned.csv', index=False)
datAll.to_csv(f'C:\\Users\\lkb32\\Desktop\\CSE6242\\Project\\data\\cancer_all_cleaned.csv', index=False)

In [31]:
datCombo.head()

,FIPS,cause_name,sex_id,sex,mx,upper,lower,YEAR,EPEST_HIGH_KG_1-METHYL CYCLOPROPENE,"EPEST_HIGH_KG_2,4-D",...,EPEST_LOW_KG_TRINEXAPAC,EPEST_LOW_KG_TRITICONAZOLE,EPEST_LOW_KG_UNICONAZOLE,EPEST_LOW_KG_VERNOLATE,EPEST_LOW_KG_VINCLOZOLIN,EPEST_LOW_KG_ZETA-CYPERMETHRIN,EPEST_LOW_KG_ZINC,EPEST_LOW_KG_ZINEB,EPEST_LOW_KG_ZIRAM,EPEST_LOW_KG_ZOXAMIDE
0,1101,Neoplasms,1,Male,360.296720,381.021791,338.226569,1992,0.0,1364.5,...,0.0,0.0,0.0,2.1,0.0,1.4,0.0,0.0,0.0,0.0
1,1101,Neoplasms,1,Male,363.438748,384.443314,341.537042,1993,0.0,1386.5,...,0.0,0.0,0.0,0.0,0.0,0.0,2.7,0.0,0.0,0.0
2,1101,Neoplasms,1,Male,370.004828,391.405199,347.508070,1994,0.0,2216.1,...,0.0,0.0,0.0,0.0,0.0,100.9,0.0,0.0,0.0,0.0
3,1101,Neoplasms,1,Male,369.583463,391.355610,347.779270,1995,0.0,834.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1101,Neoplasms,1,Male,362.898587,384.020331,340.370293,1996,0.0,1482.9,...,0.0,0.0,0.0,5.4,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
datCombo['cause_name'].unique()

array(['Neoplasms', 'Lip and oral cavity cancer', 'Nasopharynx cancer',
       'Other pharynx cancer', 'Esophageal cancer', 'Stomach cancer',
       'Colon and rectum cancer', 'Liver cancer',
       'Gallbladder and biliary tract cancer', 'Pancreatic cancer',
       'Larynx cancer', 'Tracheal, bronchus, and lung cancer',
       'Malignant skin melanoma', 'Non-melanoma skin cancer',
       'Breast cancer', 'Cervical cancer', 'Uterine cancer',
       'Ovarian cancer', 'Prostate cancer', 'Testicular cancer',
       'Kidney cancer', 'Bladder cancer',
       'Brain and nervous system cancer', 'Thyroid cancer',
       'Mesothelioma', 'Hodgkin lymphoma', 'Non-Hodgkin lymphoma',
       'Multiple myeloma', 'Leukemia', 'Acute lymphoid leukemia',
       'Chronic lymphoid leukemia', 'Acute myeloid leukemia',
       'Chronic myeloid leukemia', 'Other neoplasms'], dtype=object)

In [35]:
datCombo['YEAR'].unique()

array([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014])

In [45]:
datCombo.columns

Index(['FIPS', 'cause_name', 'sex_id', 'sex', 'mx', 'upper', 'lower', 'YEAR',
       'EPEST_HIGH_KG_1-METHYL CYCLOPROPENE', 'EPEST_HIGH_KG_2,4-D',
       ...
       'EPEST_LOW_KG_TRINEXAPAC', 'EPEST_LOW_KG_TRITICONAZOLE',
       'EPEST_LOW_KG_UNICONAZOLE', 'EPEST_LOW_KG_VERNOLATE',
       'EPEST_LOW_KG_VINCLOZOLIN', 'EPEST_LOW_KG_ZETA-CYPERMETHRIN',
       'EPEST_LOW_KG_ZINC', 'EPEST_LOW_KG_ZINEB', 'EPEST_LOW_KG_ZIRAM',
       'EPEST_LOW_KG_ZOXAMIDE'],
      dtype='object', length=1032)